## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [39]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Kapaa,US,22.08,-159.32,82.00,88,90,6.93,moderate rain
1,1,Avarua,CK,-21.21,-159.78,77.00,78,92,8.05,overcast clouds
2,2,Busselton,AU,-33.65,115.33,57.99,57,0,8.14,clear sky
3,3,Ushuaia,AR,-54.80,-68.30,42.80,75,75,28.86,light rain
4,4,Eyl,SO,7.98,49.82,80.31,76,0,8.48,clear sky


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min_temp = int(input("Please enter your desired minimum temperature in F"))
user_max_temp = int(input("Please enter your desired maximum temperature in F"))

In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df_filtered = city_data_df.loc[city_data_df['Max Temp'] > user_min_temp]
city_data_df_filtered = city_data_df_filtered.loc[city_data_df_filtered['Max Temp'] < user_max_temp]
print("Temperature filter yielded " + str(len(city_data_df_filtered)) + "/" + str(len(city_data_df)) + " results.")

Temperature filter yielded 271/685 results.


In [42]:
# 4a. Determine if there are any empty rows.
city_data_df_filtered.count()

City_ID                271
City                   271
Country                267
Lat                    271
Lng                    271
Max Temp               271
Humidity               271
Cloudiness             271
Wind Speed             271
Weather Description    271
dtype: int64

In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df_filtered = city_data_df_filtered.dropna()
city_data_df_filtered.count()

City_ID                267
City                   267
Country                267
Lat                    267
Lng                    267
Max Temp               267
Humidity               267
Cloudiness             267
Wind Speed             267
Weather Description    267
dtype: int64

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df_filtered[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.00,overcast clouds,-21.21,-159.78,
7,Sao Joao Da Barra,BR,75.20,broken clouds,-21.64,-41.05,
11,Bredasdorp,ZA,60.80,overcast clouds,-34.53,20.04,
15,Tezu,IN,73.40,light rain,27.92,96.17,
16,Port Alfred,ZA,61.77,broken clouds,-33.59,26.89,
17,Arraial Do Cabo,BR,73.40,scattered clouds,-22.97,-42.03,
21,Los Llanos De Aridane,ES,75.20,few clouds,28.66,-17.92,
23,Atambua,ID,73.13,clear sky,-9.11,124.89,
25,Carnarvon,AU,69.80,overcast clouds,-24.87,113.63,
29,Dekar,BW,76.23,clear sky,-21.53,21.93,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [49]:
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.00,overcast clouds,-21.21,-159.78,Paradise Inn
7,Sao Joao Da Barra,BR,75.20,broken clouds,-21.64,-41.05,Pousada Porto De Canoas
11,Bredasdorp,ZA,60.80,overcast clouds,-34.53,20.04,Bredasdorp Country Manor
15,Tezu,IN,73.40,light rain,27.92,96.17,Circuit House
16,Port Alfred,ZA,61.77,broken clouds,-33.59,26.89,The Halyards Hotel
...,...,...,...,...,...,...,...
672,Chipinge,ZW,62.60,clear sky,-20.19,32.62,Mutare Club
679,Ilo,PE,63.63,broken clouds,-17.64,-71.34,Torreblanca Hotel
682,Hirapur,IN,73.35,light rain,21.53,79.77,GUDRI VILLAGE
683,Kiama,AU,60.01,overcast clouds,-34.68,150.87,The Sebel Kiama Harbourside


In [51]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df = hotel_df.dropna(axis=0,how='any')
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.00,overcast clouds,-21.21,-159.78,Paradise Inn
7,Sao Joao Da Barra,BR,75.20,broken clouds,-21.64,-41.05,Pousada Porto De Canoas
11,Bredasdorp,ZA,60.80,overcast clouds,-34.53,20.04,Bredasdorp Country Manor
15,Tezu,IN,73.40,light rain,27.92,96.17,Circuit House
16,Port Alfred,ZA,61.77,broken clouds,-33.59,26.89,The Halyards Hotel
...,...,...,...,...,...,...,...
672,Chipinge,ZW,62.60,clear sky,-20.19,32.62,Mutare Club
679,Ilo,PE,63.63,broken clouds,-17.64,-71.34,Torreblanca Hotel
682,Hirapur,IN,73.35,light rain,21.53,79.77,GUDRI VILLAGE
683,Kiama,AU,60.01,overcast clouds,-34.68,150.87,The Sebel Kiama Harbourside


In [53]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "C:/Users/golfz/Documents/GitHub/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))